# Recession University Towns

We have following data for this project:

* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this project, we wil only look at GDP data from the first quarter of 2000 onward.


Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

In [1]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind
from scipy import stats

In [2]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the univesity_towns.txt.
    The format of the DataFrame should be DataFrame([["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"]],
    columns=["State", "RegionName"])
    
    The following cleaning needs to be done:
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline charater '\n'
    '''
    
    univ_towns = open('data/university_towns.txt', encoding='utf-8').readlines()
    
    # now State names are followed by [edit] so let's extract state names
    states = [line.split('[edit]')[0] for line in univ_towns if '[edit]' in line]
    
    # note that the  University towns are followed by ' ('
    towns = [x.split(' (')[0] for x in univ_towns]
    
    # Let's remove states names from the beginning
    towns = '\n'.join(towns)
    
    # Each state is followed by a pattern '[edit]\n'
    states_pattern = '\[edit\]\n|'.join(states) + '\[edit\]\n'
    
    # now split on states_pattern characters
    towns = re.split(states_pattern, towns)[1:]
    
    # now each element of our list contains all the college towns in a state, separated by
    # a '\n'. We will need to format the list of strings above as a DataFrame, so we will 
    # build a list of dictionaries with key value pairs corresponding to (state, regionname)
    universities = []
    for state in range(len(states)):
        for school in towns[state].split('\n'):
            if school != '\n' and school != '':
                universities.append({"State" : states[state], "RegionName" : school})
    
    df = pd.DataFrame(universities)
    
    return df

In [3]:
get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


In [4]:
def process_recession_file():
    recession = pd.read_excel('data/gdplev.xls', skiprows=5).iloc[2:, [4,6]]
    
    # Now the headers are "Unnamed: 4" and "GDP in billions........". Let's change them 
    recession = recession.rename(columns = {'Unnamed: 4' : 'Quarter',
                                           'GDP in billions of chained 2009 dollars.1' : 'GDP (in billions)'})
    
    # The start of recession is the first quarter of at least 2 quarters of GDP decline.
    # There are two ways to determine recesssion:
    # First method is to use .diff() method and then loop through to find out two consecutive GDP decline
    # Second method is to take the next two GDPs and put it in column next to it.
    # We are calculating recession from 2000q1 onwards
    recession = recession[recession.Quarter > '1999q4']
    recession = recession.set_index(['Quarter'])
    recession['Previous Quarter GDP'] = [np.NaN] + list(recession['GDP (in billions)'].iloc[:-1])
    recession['Previous Two Quarters GDP'] = 2*[np.NaN] + list(recession['GDP (in billions)'].iloc[:-2])
    recession['Previous Three Quarters GDP'] = 3*[np.NaN] + list(recession['GDP (in billions)'].iloc[:-3])
    recession['Previous Four Quarters GDP'] = 4*[np.NaN] + list(recession['GDP (in billions)'].iloc[:-4])
    recession['Next Quarter GDP'] = list(recession['GDP (in billions)'].iloc[1:]) + [np.NAN] 
    
    return recession

In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    recession = process_recession_file()
    
    # Now let's find the GDP in decline
    recession['Start'] = ((recession['GDP (in billions)'] < recession['Previous Quarter GDP']) \
    & (recession['GDP (in billions)'] > recession['Next Quarter GDP'])) 
    
    # Now let's track when the recession begins
    recession_start = recession[recession['Start'] == True]
    
    return recession_start.index[0]

get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession = process_recession_file()
    # Now let's find the GDP in increase after recession
    recession['End'] = (recession['Previous Four Quarters GDP'] > recession['Previous Three Quarters GDP']) \
    & (recession['Previous Three Quarters GDP'] > recession['Previous Two Quarters GDP']) \
    & (recession['Previous Two Quarters GDP'] < recession['Previous Quarter GDP']) \
    & (recession['Previous Quarter GDP'] < recession['GDP (in billions)']) 
    
    
    # Now let's track when the recession ends
    recession_end = recession[recession['End'] == True]
    return recession_end.index[0]

get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    df = process_recession_file()
    recession_begin = get_recession_start()
    recession_end = get_recession_end()
    # now extract the part of dataframe where recession begins and ends
    my_df = df.loc[recession_begin:recession_end]
    # now find the minimum in that block
    recession_bottom = my_df[my_df['GDP (in billions)'] == my_df['GDP (in billions)'].min()] 
    
    return recession_bottom.index[0]

get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.read_csv('data/City_Zhvi_AllHomes.csv')
    # we have data starting 1996 and ending 2016-08. But we are using data only since 2000q1
    
    for year in range(2000, 2017):
        for quarter in range(1,5):
            if quarter == 4 and year == 2016: 
                break
            col_name = '{0}q{1}'.format(year,quarter)
            start_month = (quarter-1)*3 + 1
            end_month = quarter*3
            start_column = '{0}-{1:02d}'.format(year,start_month)
            end_column = '{0}-{1:02d}'.format(year,end_month)
            if year == 2016 and quarter == 3:
                col_name = '2016q3'
                start_month = 7
                end_month = 8
                start_column = '{0}-{1:02d}'.format(year,start_month)
                end_column = '{0}-{1:02d}'.format(year,end_month)
            #print('Start_column :', start_column, '\tEnd_column :', end_column)
            housing_data = df.loc[:,start_column:end_column]
            df[col_name] = housing_data.mean(axis = 1)
    
    
    # Now we need to drop the columns other than means of 2000q1 to 2016q3
    drop_start = df.columns.get_loc('1996-04')
    drop_end = df.columns.get_loc('2016-08')
    df.drop(df.columns[drop_start:drop_end+1], axis=1, inplace=True)
    
    # Now change the two letter state names to full names
    # Use this dictionary to map state names to two letter acronyms
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 
              'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 
              'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 
              'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 
              'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 
              'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 
              'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 
              'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 
              'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 
              'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 
              'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 
              'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 
              'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 
              'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 
              'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 
              'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 
              'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 
              'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 
              'VA': 'Virginia'}
            
    df['State'] = df['State'].apply(lambda x: states[x])
    df = df.set_index(['State', 'RegionName'])  
    
    # Now delete all the other craps in the first four columns
    df.drop(df.columns[0:4], axis=1, inplace=True) 
    return df

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [9]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # Now call the previous functions and get the data we need for hypothesis testing
    towns = get_list_of_university_towns()
    rec_start = get_recession_start()
    rec_end = get_recession_end()
    rec_bottom = get_recession_bottom()
    housing_data = convert_housing_data_to_quarters()
    
    # first we need to set index for the university towns, because in 
    # convert_housing_data_to_quarters we set the index to State and RegionName
    towns = towns.set_index(['State', 'RegionName'])
    
    # we need to select housing data only for the period of recession begin to end
    housing_price_start = housing_data[rec_start]
    housing_price_end = housing_data[rec_end]
    
    # Now take the ratio
    housing_data['ratio'] = housing_price_start/housing_price_end
    
    # while doing t-test, it seems it gives error if there is any NaN present.
    # So we need to drop those NaNs
    college_town = housing_data.loc[list(towns.index)]['ratio'].dropna()
    
    # Ok now we got our college towns. We need to find non-college towns
    # If we take everything that is not in college town list, that is non-college town
    # First let's find the index of non-college towns
    non_college_town_index = set(housing_data.index) - set(college_town.index)
    non_college_town = housing_data.loc[list(non_college_town_index),:]['ratio'].dropna()
    
    stat, pvalue = tuple(stats.ttest_ind(college_town,non_college_town))
    
    # so for the fist part of tuple (different, p, better):
    different = pvalue < 0.05 # (if p-value is less than 0.05, we reject null hypothesis)
    
    # for the second part we already have p-value
    # for the third part:
    value = stat < 0
    better = ['non-university town', 'university town']

    return (different, pvalue, better[value])

run_ttest()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:36: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:54: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index


(True, 0.002202938784587435, 'university town')